다시 딥러닝으로 돌아와서 RNN 을 써보자 

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json

In [25]:
# 그래프 그리기 위한 함수

def plot_graph(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string], '')
    plt.xlabel('Epochs')
    plt.ylabel(string)
    plt.legend([string, 'val_' + string])
    plt.show()

In [3]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
DATA_CONFIGS = 'data_configs.json'

In [4]:
# 데이터 불러오기
train_input = np.load(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'rb'))
train_label = np.load(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

In [5]:
print(train_input.shape)
print(train_label.shape)

(25000, 174)
(25000, 174)


In [14]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

model_name = 'rnn_classifier_en'
BATCH_SIZE =128
NUM_EPOCHS = 5
VALID_SPLIT = 0.1
MAX_LEN = train_input.shape[1]

# 하이퍼 파라미션 한번에 담기

kargs = {'model_name': model_name,
        'vocab_size': prepro_configs['vocab_size'],
        'embedding_dimension': 100,
        'dropout_rate' : 0.2, 
        'lstm_dimension': 150, 
        'dense_dimension' : 150,
        'output_dimension' : 1}

### 모델 선언
- LSTM 각 단계에서 현재 정보는 input state, 이전 정보는 hidden state 라 함  
- LSTM 의 매개변수 중 return_sequences 은 hidden state 의 반환값을 결정하는데  
- false 일 경우 predict 시 마지막 hidden state만 출력됨  
- true 일 경우 각 time step 별 모든 hidden state 를 출력함  
- 이전의 기록들이 다음 레이어로 들어가기 때문에 LSTM레이어끼리 엮기 위해서는 return_sequence를 사용하여야 함 

In [20]:
# 모델 만들기 
# subclassic 사용
# RNNClassifier 는 tf.keras의 모델을 상속받는다. 두개의 인자가 필요하다. 셀프와 
# 딕셔러니 형태는 받을 떄 ** 를 붙여줘야 한다 
# def 는 정의. 실제 층을 쌓는 것은 def call

class RNNClassifier(tf.keras.Model): 
    def __init__(self, **kargs):
        super(RNNClassifier, self).__init__(name=kargs['model_name'])
        self.embedding = layers.Embedding(input_dim=kargs['vocab_size'],
                                         output_dim=kargs['embedding_dimension'])
        self.lstm_1_layer = layers.LSTM(kargs['lstm_dimension'],return_sequences=True)
        self.lstm_2_layer = layers.LSTM(kargs['lstm_dimension']) 
        self.dropout = layers.Dropout(kargs['dropout_rate'])
        self.fc1 = layers.Dense(kargs['dense_dimension'], activation=tf.keras.activations.tanh)
        self.fc2 = layers.Dense(kargs['output_dimension'], activation = tf.keras.activations.sigmoid)
        
    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.lstm_1_layer(x)
        x = self.lstm_2_layer(x)
        x = self.dropout(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x 

In [21]:
# LSTM 레이어를 2개 이상 격자로 쓸 때는 return_seqyebces=True로 감싸줘야 한다 

In [22]:
model = RNNClassifier(**kargs)
model.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])

In [23]:
earlystop_callback = EarlyStopping(monitor='val_accuacy', min_delta=0.0001, patience=3)
checkpoint_path = DATA_OUT_PATH + model_name + '/weights.h5'
checkpoint_dir = os.path.dirname(checkpoint_path)
if os.path.exists(checkpoint_dir):
    print('{} -- Folder already exists \n'.format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok =True)
    print('{} -- Folder create complete \n'.format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1,
                              save_bset_only=True, save_weights_only=True)


./data_out/rnn_classifier_en -- Folder already exists 



In [24]:
history = model.fit(train_input,train_label, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=VALID_SPLIT,
                    callbacks=[earlystop_callback, cp_callback])

Train on 22500 samples, validate on 2500 samples
Epoch 1/5
  128/22500 [..............................] - ETA: 1:27WARNING:tensorflow:Early stopping conditioned on metric `val_accuacy` which is not available. Available metrics are: 

Epoch 00001: saving model to ./data_out/rnn_classifier_en/weights.h5


ValueError: logits and labels must have the same shape ((None, 1) vs (None, 174))